## Progress Log: To‑Do List Manager

**Last Updated:** July 2, 2025  
**Author:** Jerick C. Jualo

I’ve been steadily enhancing my To‑Do List Manager. Here’s what I’ve added, refactored, and polished so far:

---

### 1. Authentication & Account Management (`classes_functs.py`)

- **Persistent Account Registry**  
  - Introduced `Account.LIST_ACC_ID`, `Account.ALL_ACC`, `Account.NO_ACCOUNTS`, and `Account.DETAILS_ACCOUNT` to track every user I create.

- **Login Returns the Account Object**  
  - Updated `log_in()` so it now returns the actual `Account` instance (`logged_acc`) instead of just `True`.

- **Per‑Account Task Store**  
  - Switched from a one‑slot dict to a proper `self.tasks` dict keyed by each task’s `task_id`.  
  - Renamed the store from `list_tasks` to `tasks` for clarity.

- **Account‑Scoped Task Factory**  
  - Moved `create_task()` into the `Account` class. Now it:
  1. Prompts for task name, description, and status  
  2. Instantiates `Task(..., owner_id=self.acc_id)`  
  3. Inserts the new task into `self.tasks[new_task.task_id]`  
  4. Returns the `Task` object for further actions

- **Account Detail & Task Views**  
  - `view_tasks()` prints a neat list of all tasks for the current account (ID, name, status, date created).  
  - `full_acc_details()` shows account metadata plus a full dump of each `Task` via its `__repr__`.

---

### 2. Task Model & Behaviors (`tasks.py`)

- **One‑to‑Many Ownership**  
  - Added `self.owner_id` in `Task.__init__` so every task knows which `acc_id` created it.  
  - Maintained global registries (`Task.ALL_TASKS`, `Task.NO_TASKS`, `Task.LIST_TASK_ID`) for analytics or bulk exports.

- **Improved String Representations**  
  - Implemented both `__repr__` (for concise debugging) and `__str__` (for multi‑line, pretty‑printed display).

- **Editable Fields with Confirmation**  
  - Methods `rename()`, `change_description()`, and `change_status()` now:  
    1. Validate input (e.g., non‑empty name, valid status)  
    2. Prompt for “yes/no” confirmation  
    3. Apply the change or cancel gracefully

- **Due‑Date Setter**  
  - Interactive `set_due_date()` with month/day validation, returning a `datetime` object for serialization.

---

### 3. Main Flow & Menuing (`main_module.py` + `menu.py`)

- **Central Entry Point**  
  - `main_module.py` handles the welcome screen, login loop, then calls `super_menu(logged_acc)` once authenticated.

- **Interactive Super‑Menu**  
  - Options include:  
    - View account details  
    - View my tasks  
    - Create a new task  
    - Edit an existing task  
    - Log out

- **Edit Workflow**  
  - When choosing “Edit a Task”:  
    1. Reprint current tasks  
    2. Prompt for `task_id`  
    3. Call `edit_task(logged_acc, task_id)`, which dispatches to `task.rename()`, `task.change_description()`, or `task.change_status()` as needed

---

### 4. What’s Next

1. **Deleting & Archiving Tasks**  
   - Add `delete_task(task_id)` to remove tasks from both `self.tasks` and `Task.ALL_TASKS`.

2. **Editing Due Dates from the Menu**  
   - Expose `task.set_due_date()` in the menu for adding or changing deadlines interactively.

3. **Persistence Layer**  
   - Implement `serialize()` / `deserialize()` on both classes to read/write JSON or CSV at startup/shutdown.


In [ ]:
# FROM MAIN_MODULE

from classes_functs import log_in
from classes_functs import Account
from menu import super_menu

# Introduction

print("""welcome to my To-Do List Manager
      
Coded by: Jerick C. Jualo""")

print("\n=======================================================================================================================================================")
print("Welcome to the To-Do Task Manager!")
print("=======================================================================================================================================================")
print("This program will help you track your goals and weekly schedule for Python, SQL Learning, and Math Study.")
print("Also, it will help you create your tasks for today and check the status of your tasks.")
print("Please log in to continue.")
print("=======================================================================================================================================================\n")


# Ensure the user is logged in before proceeding 

logged_acc = None
# Loop until the user successfully logs in
while not logged_acc:
    logged_acc = log_in()

super_menu(logged_acc)


# After the user logs in, the super_menu function will be called to display the main menu and handle further interactions.



In [ ]:
#FROM classes_functs.py 

from datetime import datetime 
from tasks import Task

#CLASS FOR ACCOUNT
class Account:
    
    # Class Attributes
    LIST_ACC_ID: list[int] = []
    ALL_ACC: list[str] = []
    NO_ACCOUNTS: int = 0
    DETAILS_ACCOUNT = {}
        
    
    def __init__(self, username: str, password: str):

        # Username, Password, Account ID
        key = username.lower()
        self.username = username
        self.password = password
        
        next_id: int = Account.LIST_ACC_ID[-1] + 1 if Account.LIST_ACC_ID else 1

        self.acc_id: int = next_id # Owner ID on Task Class
        
        #Task related
        
        self.tasks = {}
        

        #Actions to be executed for Class Attributes
        Account.LIST_ACC_ID.append(next_id)
        Account.ALL_ACC.append(self)
        Account.NO_ACCOUNTS += 1
        Account.DETAILS_ACCOUNT[key] = password 
        


    # Instance Methods

    #Account Details Display Methods

    def display_NO_ACCOUNTS(self) -> str:
        return f"Total Number of Accounts: {Account.NO_ACCOUNTS}"
    
    
    def display_accounts(self) -> str:
        return f"Account Details: {Account.DETAILS_ACCOUNT}"

    def view_acc_id(self) -> str:
        return f"Account: {self.username} ID: {self.acc_id}"
    
    
    def full_acc_details(self) -> str:
        print(f"Account Details for {self.username}:")
        print(f"Username: {self.username}")
        print(f"Account ID: {self.acc_id}")
        print(f"Tasks: {self.tasks}")

        for x in self.tasks.values():
            print(x)
        print("==========================")
        return ""

        
    def __repr__(self) -> str:
        return f"Account(Username: '{self.username}',Password: '{self.password}', Account ID: '{self.acc_id}')"
    
    # Task Related Methods

    def view_tasks(self):
        """
        1. Display all tasks for the account.
        2. If no tasks exist, inform the user.
        3. If tasks exist, display each task's details.
        """
        
        if not self.tasks:
            print("No tasks available for this account.")
            return
        
        print(f"Tasks for Account '{self.username}':")
        for task_id, task in self.tasks.items():
            print(f"Task ID: {task_id}, Name: {task.task_name}, Status: {task.status}, Date Created: {task.date_created.strftime('%Y-%m-%d %H:%M:%S')}")

        
        
    def create_task(self):
        print("Creating a new task:" )

        # a dictionary for task status options
        status = {"1": "Not Started",
            "2": "In Progress",
            "3": "Completed",
            "4": "On Hold"
        }

        # Asking the task detials

        task_name: str = input("Enter the task name: ")
        description: str = input("Enter the task description: ")
        status_choice: str = input("""Enter the task status: 
                                (1: Not Started 
                                2: In Progress
                                3: Completed
                                4: On Hold)
                                : """)
        
        # Validate the status choice and set the task status
        if status_choice in status:
            task_status = status[status_choice]
        else:
            print("Invalid status choice. Defaulting to 'Not Started'.")
            task_status = status["1"]

        #Instantiate a new Task object

        new_task = Task(task_name, description, task_status, self.acc_id)
        
        self.tasks[new_task.task_id] = new_task

        print(f"""Task '{new_task.task_name}' created successfully with status '{new_task.status}'!
            Task_ID: {new_task.task_id}
            Description: {new_task.description}
            Date Created: {new_task.date_created.strftime('%Y-%m-%d %H:%M:%S')}
            Week Day Created: {new_task.date_created.strftime("%A")}""")
            
        new_task.day_created = new_task.date_created.strftime("%A")
        
        return new_task

    

    
    #Class Methods

    @classmethod
    def create_account(cls):

        """
    1. Loop until unique username + matching passwords
    2. Instantiate new account(...)
    3. Confirm creation, return to login flow
    """

        while True:
            
            print("Creating Account:")

            create_user: str  = input("Enter the Username: ")
    
            # Check if the username already exists
            if create_user in Account.DETAILS_ACCOUNT:
                print("Username already exists. Please try a different username.")
                continue
            
            first_password: str  = input("Enter the Password: ")
            final_password: str  = input("Confirm the Password: ")
            
            if first_password != final_password:
                print("Passwords do not match. Please try again.")
                continue
    
            print(f"Account created successfully for {create_user}!")
            return cls(create_user, final_password)
            break


#FUNCTION SECTIONS


def log_in():

    """
    1. Prompt for username/password
    2. Check against account.DETAILS_ACCOUNT
    3. On success: greet & return True
    4. On failure: call create_account() or exit
    """
    
    print("=======================================================================================================================================================")
    print("Log In:")
    log_username: str = input("\nEnter your username: ").lower()
    log_password: str = input("Enter your password: ")
    
    

    if log_username.lower() in Account.DETAILS_ACCOUNT and Account.DETAILS_ACCOUNT[log_username.lower()] == log_password:
        print(f"\nWelcome back, {log_username.title()}!")
        for acc in Account.ALL_ACC:
            if acc.username == log_username:
                global logged_acc
                logged_acc = acc
                break

        return logged_acc

    else:
        print("User is not found.")
        confirm: str  = input("\nDo you want to create an account? (y/n): ").lower()
        if confirm == "y":
            Account.create_account()
        else:
            print("Exiting the program. Please run again with the correct username.")
            exit()

In [ ]:
#FROM tasks.py

from datetime import datetime

#CLASS SECTION

class Task:

    statuses: list = ["Not Started", "In Progress", "Completed", "On Hold"]

    #Task Class Attributes
    ALL_TASKS: list = []
    NO_TASKS: int = 0
    LIST_TASK_ID: list[int] = []

    def __init__(self, task_name: str, description: str, status: str = "Not Started", owner_id: int = None):

        # Owner ID 
        self.owner_id = owner_id

        #Task ID generation
        next_id: int = Task.LIST_TASK_ID[-1] + 1 if Task.LIST_TASK_ID else 1
        self.task_id: int = next_id

        # Task details attributes
        
        self.task_name = task_name
        self.description = description
        self.status = status
        self.details = {}

        #Task date attributes
        self.date_created = datetime.now()
        self.due_date = None
        self.day_created = None
        

        # Add the new task to the list of all tasks and update the task count
        Task.ALL_TASKS.append(self)
        Task.NO_TASKS += 1
        Task.LIST_TASK_ID.append(self.task_id)
        
    def __repr__(self) -> str:
        return f"Task(Task ID: '{self.task_id}', Owner ID: '{self.owner_id}', Task Name: '{self.task_name}', Description: '{self.description}', Status: {self.status})"
    
    def __str__(self) -> str:
        return f"""
        ===========================
        Task Information
        ===========================
        Task ID: {self.task_id}
        Owner ID: {self.owner_id}

        ===========================
        Task Details:
        ===========================
        Task Name: {self.task_name}
        Description: {self.description}
        Status: {self.status}
        Date Created: {self.date_created}
        Due Date: {self.due_date}
        Day Created: {self.day_created}
        """


    def set_due_date(self):
            
            """
            1. Set the due date for the task.
            2. The user will be prompted to enter the year,
               month, and day for the due date.
            3. The month will be selected from a list of months,
               and the day must be valid for the selected month.
            4. The due date will be stored as a datetime object.
            5. If the user enters an invalid date, they will be prompted to try again
            6. The due date will be displayed to the user.
            """

            # Dictionary to map month numbers to month names 
            month_list = {"1": "January", "2": "February", "3": "March", "4": "April", "5":"May", "6": "June",
                            "7": "July", "8": "August", "9": "September", "10": "October", "11": "November", "12": "December"}
            
            # Dictionary to hold the maximum days in each month
            month_day_limit = {"January": 31, "February": 28, "March": 31, "April": 30, "May": 31, "June": 30,
                            "July": 31, "August": 31, "September": 30, "October": 31, "November": 30, "December": 31}
            
            print(f"Set the Due date of the task {self.task_name}: ")
            year = input("Enter the Year(eg. 2025): ").strip()

            while True:

                month_choice = input("""Enter the Month:
                            January - 1
                            February - 2
                            March   - 3
                            April   - 4
                            May     - 5
                            June    - 6
                            July    - 7
                            August  - 8
                            September - 9
                            October   - 10
                            November  - 11
                            December  - 12
                            : """).strip()
                
                
                
                if month_choice not in month_list:
                    print("Invalid Month Choice, Please try again. ")
                    continue
                
                month = month_list[month_choice]
                day_limit = month_day_limit[month]
                break
                    
            
            #Ensure the day not exceed the 31 day month limit
            while True:

                day = input("Enter the Day(eg. 1, 10, 25): ").strip()

                if int(day) > day_limit:
                    print("Invalid day, Please Enter a day within 31")
                    continue

                break
            
            self.due_date = datetime(int(year) if year.is_digit() else "Invalid Year Format", int(month_choice), day)
            print(f"{self.task_name} due date is now set!: {self.due_date}")

            return self.due_date
    




    def rename(self, new_task_name: str):
        """
        Change the name of the task.
        """

        if not new_task_name:
            print("Task name cannot be empty. Please provide a valid name.")
            return None

        confirmation = input(f"Are you sure you want to change the task name from {self.task_name} to {new_task_name}? (yes/no): ").strip().lower()

        if confirmation == "yes":

            self.task_name = new_task_name
            print(f"Task name changed to: {self.task_name}")
            return self.task_name
        
        else:
            print("Change Task Name Cancelled")
            return None
    



    
    def change_description(self, new_description: str):
        """
        Change the description of the task.
        """

        if not new_description:
            print("Description cannot be empty. Please provide a valid description.")
            return None
        
        confirmation = input(f"Are you sure you want to change the description of {self.task_name} to '{new_description}'? (yes/no): ").strip().lower()

        if confirmation == "yes":

            self.description = new_description
            print(f"Task description changed to: {self.description}")
            return self.description
        
        else:
            print("Change Description Cancelled")

            return None




    def change_status(self, new_status: str):

        if new_status not in Task.statuses:
            print(f"Invalid status. Please choose from: {', '.join(Task.statuses)}")
            return None
        
        confirmation = input(f"Are you sure you want to change the status of {self.task_name} to {new_status}? (yes/no): ").strip().lower()
        
        if confirmation == 'yes':
            self.status = new_status
            print(f"Status of {self.task_name} changed to: {self.status}")
            return self.status
        else:
            print("Status change cancelled.")
            return None

In [ ]:
#FROM menu.py

def super_menu(logged_acc):

    print(f"Welcome {logged_acc.username} to the To-Do Task Manager!")

    while True:
    
        print("\nPlease choose an option:")
        print("1. View Account Details")
        print("2. View Tasks")
        print("3. Create a New Task")
        print("4. Edit a Task")
        print("5. Log Out")

        choice = input("Enter your choice (1-5): ").strip()

        if choice == "1":
            print(logged_acc.full_acc_details())
        elif choice == "2":
            print(logged_acc.view_tasks())
        elif choice == "3":
            logged_acc.create_task()
        elif choice == "4":
            print(logged_acc.view_tasks())

            edit_id = input("\nEnter the Task ID you want to edit:").strip()
            if edit_id in logged_acc.tasks:
                logged_acc.edit_task(logged_acc, edit_id)
            else:
                print("Task ID not found.")
        
        elif choice == "5":
            print("Logging out...")
            break


def edit_task(logged_acc, task_id):
    """
    Edit an existing task.
    """

    task = logged_acc.tasks[task_id]


    print(f"Editing Task: {task.task_name}")

    print("Current Task Details:")
    print(f"Name: {task.task_name}")
    print(f"Description: {task.description}")
    print(f"Status: {task.status}")
    print(f"Due Date: {task.due_date.strftime('%Y-%m-%d') if task.due_date else 'No due date set'}")

    
    print("Please choose an option to edit:")
    print("1. Change Task Name")
    print("2. Change Task Description")
    print("3. Change Task Status")

    edit_choice = input("Enter your choice (1-3): ").strip()

    if edit_choice == "1":
        new_name = input("Enter the new task name: ").strip()
        task.rename(new_name)

    elif edit_choice == "2":
        new_description = input("Enter the new task description: ").strip()
        task.change_description(new_description)

    elif edit_choice == "3":
        new_status = input(f"Enter the new status for the task (current: {task.status}): ").strip()
        task.change_status(new_status)

    else:
        print("Invalid choice. Please try again.")